In [9]:
import requests
import pandas as pd
import time
import json
from collections import namedtuple
from bs4 import BeautifulSoup
import lxml.html
import glob

## Scrap members

In [2]:
MEP_Member = namedtuple('MEP_member', ['name', 'group', 'link','country','party','status'])

In [3]:
def scrap_data_MEP_members(html_string):
    soup = BeautifulSoup(html_string, 'lxml') # Parse the HTML as a string
    
    table = soup.find_all('table')[0] # Grab the first table

    rows = table.find_all('tr')
    rows = [r for r in rows if len(r)>0]
    rows = rows[1:]
    
    all_members = []
    for r in rows:
        try:
        

            link =r.find('a')['href']
            name = r.find_all("td")[0].text.strip()
            group = r.find_all("td")[1].text.strip()
            country = r.find_all("td")[2].text.strip()
            party = r.find_all("td")[3].text.strip()
            status = r.find_all("td")[4].text.strip()

            all_members.append(MEP_Member(name,group,link,country,party,status))
        except:
            print(r)

    return all_members


In [4]:
result = requests.get("https://www.votewatch.eu//en/term8-european-parliament-members.html?limit=1000")
s =scrap_data_MEP_members(result.text)
df = pd.DataFrame(s)
df['id']=df.index+1
df[df.id==386]
df.to_csv('MEP_lists.csv',encoding='utf-8')

## Scrap votes

In [5]:
def scrap_votes(member_id):
    url_votes = f"https://www.votewatch.eu/actions.php?euro_parlamentar_id={str(member_id)}&form_category=get_mep_acte&sEcho=3&iColumns=6&sColumns=&iDisplayStart=0&iDisplayLength=20000"
    result = requests.get(url_votes).json()
    df_votes = pd.DataFrame(result['all_votes'])
    df_votes['loyalty']=df_votes.euro_vot_rol_euro_grup.apply(pd.Series)['rol_af']
    df_votes['member_id']=member_id
    return df_votes

In [6]:
all_votes = pd.DataFrame()

for member in df.id.tolist()[180:]:
    print(str(member))
    
    try:
        df_votes = scrap_votes(member)
        all_votes = pd.concat([all_votes,df_votes],axis=0)
    except:
        print(f'could not scrap votes for member {str(member)}')
    if member % 10 ==0:
        print(f'save point on {str(member)}')
        all_votes.to_csv(f'MEP_votes_{str(member)}.csv')
        all_votes = pd.DataFrame()  
        
all_votes.to_csv(f'MEP_votes_last.csv')

181
182
183
184
185
186
187
188
189
190
save point on 190
191
192
193
194
195
196
197
198
199
200
save point on 200
201
202
203
204
205
206
207
208
209
210
save point on 210
211
212
213
214
215
216
217
218
219
220
save point on 220
221
222
223
224
225
226
227
228
229
230
save point on 230
231
232
233
234
235
236
237
238
239
240
save point on 240
241
242
243
244
245
246
247
248
249
250
save point on 250
251
252
253
254
255
256
257
258
259
260
save point on 260
261
262
263
264
265
266
267
268
269
270
save point on 270
271
272
273
274
275
276
277
278
279
280
save point on 280
281
282
283
284
285
286
287
288
289
290
save point on 290
291
292
293
294
295
296
297
298
299
300
save point on 300
301
302
303
304
305
306
307
308
309
310
save point on 310
311
312
313
314
315
316
317
318
319
320
save point on 320
321
322
323
324
325
326
327
328
329
330
save point on 330
331
332
333
334
335
336
337
338
339
340
save point on 340
341
342
343
344
345
346
347
348
349
350
save point on 350
351
352
353
35

In [9]:
all_votes.to_csv(f'MEP_votes_last.csv')


## read all votes files

In [80]:
all_votes_files =  [f for f in glob.glob('MEP_votewatch/votes/*.csv', recursive=True)]

## Create votes domains table

In [71]:
df = pd.read_csv(all_votes_files[2])

In [46]:
df_domains = df[['euro_domeniu_id','euro_domeniu_nume']].drop_duplicates().sort_values('euro_domeniu_id').set_index('euro_domeniu_id')

In [48]:
df_domains.to_csv('MEP_votes_domains.csv')

## Create actes tables

In [69]:
df_actes = df[['act_nume_full','euro_act_id']].drop_duplicates()
df_actes['acte_title'] = df_actes.act_nume_full.str.extract('<a[^>]*>(.*?)</a>')
df_actes['acte_title'] = df_actes['acte_title'].str.replace('<b>','').str.replace('<[^<]+?>', '')
df_actes['link']= df_actes.act_nume_full.str.extract('href=[\'"]?([^\'" >]+)')
df_actes.to_csv('MEP_votes_actes.csv')

## Filter votes on id

In [81]:
def filter_votes_data(file):
    df = pd.read_csv(file)
    df_votes = df[['euro_act_id','euro_domeniu_id','euro_vot_valoare_text','mysql_data_text','loyalty','member_id']]
    return df_votes
    

In [84]:
df_all_votes_light = pd.DataFrame()

for f in all_votes_files:
    df_light_votes = filter_votes_data(f)
    df_all_votes_light = pd.concat([df_all_votes_light,df_light_votes],axis=0)
    
    

In [87]:
df_all_votes_light.to_csv('all_votes.csv')

## Votes definition

In [6]:
votes_definitions = {'M':'Documented Absence','N':'Did not vote',"-":"Against","+":"In favour","A":"Absent","0":"Abstained"}

In [22]:
df_votes_definitions = pd.DataFrame.from_dict(votes_definitions,orient='index').reset_index()
df_votes_definitions.columns = ['vote_id','vote_name']
df_votes_definitions.to_csv('MEP_votes_definitions.csv', index=False)